In [1]:
# %matplotlib inline
import numpy as np
import k3d
from k3d.colormaps import matplotlib_color_maps
import pathlib
import matplotlib.pyplot as plt
import matplotlib
import imageio
import svox2
import torch
import tqdm
import open3d as o3d

/home/tw554/plenoxels/svox2/utils.py:39: UserWarning: CUDA extension svox2.csrc could not be loaded! Operations will be slow.
Please do not import svox in the svox2 source directory.
  warn("CUDA extension svox2.csrc could not be loaded! " +


In [2]:
path = "/home/tw554/plenoxels/opt/ckpt/dtu_scan63/no_surface_init_long_init/pts.npy"
pts = np.load(path)

pts.shape

(1316532, 3)

In [3]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(pts)

# downsample point cloud
voxel_size = 0.001
downpcd = pcd.voxel_down_sample(voxel_size=voxel_size)
radius_normal = voxel_size * 2

print(np.asarray(downpcd.points).shape)

# downpcd.estimate_normals(
#     search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal,max_nn=30)
#     )



(4637485, 3)


In [5]:
plt_points = k3d.points(np.asarray(downpcd.points),
                        point_size=0.0001,
                        shader="flat")

plot = k3d.plot(grid_visible=True,
                camera_auto_fit=True)
plot += plt_points
plot.display()

Output()

In [6]:
downpcd.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal,max_nn=30)
    )



run Poisson surface reconstruction
[Open3D DEBUG] Input Points / Samples: 4637485 / 983080
[Open3D DEBUG] #   Got kernel density: 0.490167 (s), 1473.95 (MB) / 1473.95 (MB) / 1603 (MB)
[Open3D DEBUG] #     Got normal field: 3.58199 (s), 1688.03 (MB) / 1688.03 (MB) / 1688 (MB)
[Open3D DEBUG] Point weight / Estimated Area: 9.918878e-07 / 4.599864e+00
[Open3D DEBUG] #       Finalized tree: 2.71658 (s), 1839.43 (MB) / 1839.43 (MB) / 1839 (MB)
[Open3D DEBUG] #  Set FEM constraints: 2.73046 (s), 1733.34 (MB) / 1839.43 (MB) / 1839 (MB)
[Open3D DEBUG] #Set point constraints: 1.10605 (s), 1733.34 (MB) / 1839.43 (MB) / 1839 (MB)
[Open3D DEBUG] Leaf Nodes / Active Nodes / Ghost Nodes: 6406338 / 7310000 / 11529
[Open3D DEBUG] Memory Usage: 1733.344 MB
[Open3D DEBUG] # Linear system solved: 5.9224 (s), 1955.69 (MB) / 1955.69 (MB) / 1955 (MB)
[Open3D DEBUG] Got average: 0.252813 (s), 1709.45 (MB) / 1955.69 (MB) / 1955 (MB)
[Open3D DEBUG] Iso-Value: 5.005371e-01 = 2.321233e+06 / 4.637485e+06
Cycle[0] 

[WARNING] /home/runner/work/Open3D/Open3D/3rdparty/PoissonRecon/PoissonRecon/Src/FEMTree.IsoSurface.specialized.inl (Line 1858)
          Extract
          bad average roots: 4


[Open3D DEBUG] #          Total Solve:      36.1 (s),    2408.6 (MB)


In [ ]:
mesh_recon_type = 'rolling_ball'

if mesh_recon_type == 'rolling_ball':
    # estimate radius for rolling ball
    print('run rolling ball surface reconstruction')
    distances = downpcd.compute_nearest_neighbor_distance()
    avg_dist = np.mean(distances)
    radius = 1.5 * avg_dist   
    print(f'Radius: {radius}')

    mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
               downpcd,
               o3d.utility.DoubleVector([radius, radius * 2]))
elif mesh_recon_type == 'poisson':
    print('run poisson surface reconstruction')
    with o3d.utility.VerbosityContextManager(
            o3d.utility.VerbosityLevel.Debug) as cm:
        mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
            downpcd, depth=9)

In [7]:
np.asarray(mesh.vertices)
np.asarray(mesh.triangles)

array([[      0,       1,      38],
       [     38,      37,       0],
       [      1,       2,      39],
       ...,
       [2363452, 2363718, 2363668],
       [2363719, 2363455, 2363668],
       [2363720, 2363455, 2363719]], dtype=int32)

In [ ]:
plt_points = k3d.mesh(np.asarray(mesh.vertices),
                      np.asarray(mesh.triangles),
                      shader="flat")

plot = k3d.plot(grid_visible=True,
                camera_auto_fit=True)
plot += plt_points
plot.display()

In [3]:
import numpy as np
import open3d as o3d
import sklearn.neighbors as skln
from tqdm import tqdm
from scipy.io import loadmat
import multiprocessing as mp
import argparse
import k3d

def sample_single_tri(input_):
    n1, n2, v1, v2, tri_vert = input_
    c = np.mgrid[:n1+1, :n2+1]
    c += 0.5
    c[0] /= max(n1, 1e-7)
    c[1] /= max(n2, 1e-7)
    c = np.transpose(c, (1,2,0))
    k = c[c.sum(axis=-1) < 1]  # m2
    q = v1 * k[:,:1] + v2 * k[:,1:] + tri_vert
    return q

def write_vis_pcd(file, points, colors):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.colors = o3d.utility.Vector3dVector(colors)
    o3d.io.write_point_cloud(file, pcd)
    

In [7]:
parser = argparse.ArgumentParser()
parser.add_argument('--pts_dir', type=str, default='/home/tw554/plenoxels/opt/ckpt/dtu_scan63/no_surface_init_long_init/')
parser.add_argument('--scan', type=int, default=63)
parser.add_argument('--dataset_dir', type=str, default='/home/tw554/plenoxels/data/dtu_eval/SampleSet/MVS Data')
parser.add_argument('--downsample_density', type=float, default=0.2)
parser.add_argument('--patch_size', type=float, default=60)
parser.add_argument('--max_dist', type=float, default=20)
parser.add_argument('--visualize_threshold', type=float, default=10)
args = parser.parse_args(args=[])

thresh = args.downsample_density

pbar = tqdm(total=8)
pbar.set_description('read data pcd')
data_pcd = np.load(f'{args.pts_dir}/pts.npy')
data_down = data_pcd

# # rescale pts
# cam = np.load(f'/home/tw554/plenoxels/data/dtu/dtu_scan{args.scan}/cameras_large.npz')
# data_pcd = np.load(f'{args.pts_dir}/pts.npy')
# data_pcd = data_pcd * cam['scale_mat_0'][0,0] + cam['scale_mat_0'][:3,3][None]
# data_pcd = data_pcd[:, :3]

pbar.update(1)
pbar.set_description('random shuffle pcd index')
shuffle_rng = np.random.default_rng()
shuffle_rng.shuffle(data_pcd, axis=0)


nn_engine = skln.NearestNeighbors(n_neighbors=1, radius=thresh, algorithm='kd_tree', n_jobs=-1)


random shuffle pcd index:  12%|█████████▎                                                                | 1/8 [00:00<00:00, 84.74it/s]

In [3]:
pbar.update(1)
pbar.set_description('downsample pcd')

nn_engine.fit(data_pcd)
rnn_idxs = nn_engine.radius_neighbors(data_pcd, radius=thresh, return_distance=False)
mask = np.ones(data_pcd.shape[0], dtype=np.bool_)
for curr, idxs in enumerate(rnn_idxs):
    if mask[curr]:
        mask[idxs] = 0
        mask[curr] = 1
data_down = data_pcd[mask]

downsample pcd:  25%|█████████████████████████▌                                                                            | 2/8 [00:24<01:13, 12.25s/it]

In [6]:
stl_pcd = o3d.io.read_point_cloud(f'{args.dataset_dir}/Points/stl/stl{args.scan:03}_total.ply')
stl = np.asarray(stl_pcd.points)

plt_points = k3d.points(data_pcd[0:-1:10,:],
                        color=0xff0000,
                        point_size=0.0001,
                        shader="flat")

plt_points_2 = k3d.points(stl[0:-1:10,:],
                        color=0x00ff00,
                        point_size=0.0001,
                        shader="flat")

plot = k3d.plot(grid_visible=False,
                camera_auto_fit=True)
plot += plt_points
plot += plt_points_2
plot.display()

/home/tw554/.conda/envs/voxel_sdf/lib/python3.8/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


Output()

In [3]:
stl_pcd = o3d.io.read_point_cloud(f'{args.dataset_dir}/Points/stl/stl{args.scan:03}_total.ply')
stl = np.asarray(stl_pcd.points)

plt_points = k3d.points(np.load(f'{args.pts_dir}/pts_2.npy')[0:-1:10,:],
                        color=0xff0000,
                        point_size=0.0001,
                        shader="flat")

plt_points_2 = k3d.points(stl[0:-1:10,:],
                        color=0x00ff00,
                        point_size=0.0001,
                        shader="flat")

plot = k3d.plot(grid_visible=False,
                camera_auto_fit=True)
plot += plt_points
plot += plt_points_2
plot.display()

/home/tw554/.conda/envs/voxel_sdf/lib/python3.8/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


Output()

(383298, 3)

In [4]:
data_pcd = np.load(f'{args.pts_dir}/pts.npy')

lb = np.array([-300,-200,600])
ub = np.array([-50,0,800])

mask = (data_pcd > lb).all(axis=-1) & (data_pcd < ub).all(axis=-1)
pts = data_pcd[mask]
pts.shape

plt_points = k3d.points(pts,
                        color=0xff0000,
                        point_size=0.001,
                        shader="flat")


plot = k3d.plot(grid_visible=True,
                camera_auto_fit=True)
plot += plt_points
plot.display()

Output()

In [6]:
data_pcd = np.load(f'{args.pts_dir}/pts_2.npy')

lb = np.array([-300,-200,600])
ub = np.array([-50,0,800])

mask = (data_pcd > lb).all(axis=-1) & (data_pcd < ub).all(axis=-1)
pts = data_pcd[mask]
# pts = data_pcd
pts.shape



plt_points = k3d.points(pts,
                        color=0xff0000,
                        point_size=0.001,
                        shader="flat")


plot = k3d.plot(grid_visible=True,
                camera_auto_fit=True)
plot += plt_points
plot.display()

Output()

In [7]:
data_pcd = np.load(f'{args.pts_dir}/pts_3.npy')

lb = np.array([-300,-200,600])
ub = np.array([-50,0,800])

mask = (data_pcd > lb).all(axis=-1) & (data_pcd < ub).all(axis=-1)
pts = data_pcd[mask]
# pts = data_pcd
pts.shape



plt_points = k3d.points(pts,
                        color=0xff0000,
                        point_size=0.001,
                        shader="flat")


plot = k3d.plot(grid_visible=True,
                camera_auto_fit=True)
plot += plt_points
plot.display()

Output()

In [10]:
data_pcd = np.load(f'{args.pts_dir}/pts_3.npy')

lb = np.array([-240,-200,640])
ub = np.array([-160,0,680])

mask = (data_pcd > lb).all(axis=-1) & (data_pcd < ub).all(axis=-1)
pts = data_pcd[mask]
# pts = data_pcd
pts.shape



plt_points = k3d.points(pts,
                        color=0xff0000,
                        point_size=0.001,
                        shader="flat")


plot = k3d.plot(grid_visible=True,
                camera_auto_fit=True)
plot += plt_points
plot.display()

Output()

In [14]:
from opt.util.dataset import datasets
dset = datasets["auto"](
               "data/dtu/dtu_scan63",
               split="train",
               device="cuda",
               factor=1,
               n_images=-1)


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 49/49 [00:05<00:00,  9.69it/s]


 Generating rays, scaling factor 1


/home/tw554/.conda/envs/voxel_sdf/lib/python3.8/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755903507/work/aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [15]:
dset.rescale2world(pts)

array([[-0.32334685, -0.21004248,  0.09764756],
       [-0.3296334 , -0.21021104,  0.09848092],
       [-0.328991  , -0.21008903,  0.09807844],
       ...,
       [-0.31540674, -0.02715903,  0.07484099],
       [-0.31482303, -0.02720326,  0.07537636],
       [-0.31443095, -0.02719915,  0.07541216]], dtype=float32)

In [8]:
pbar.update(1)
pbar.set_description('masking data pcd')
obs_mask_file = loadmat(f'{args.dataset_dir}/ObsMask/ObsMask{args.scan}_10.mat')
ObsMask, BB, Res = [obs_mask_file[attr] for attr in ['ObsMask', 'BB', 'Res']]
BB = BB.astype(np.float32)

data_down = data_pcd

patch = args.patch_size
patch = 600
inbound = ((data_down >= BB[:1]-patch) & (data_down < BB[1:]+patch*2)).sum(axis=-1) ==3
data_in = data_down[inbound]

data_grid = np.around((data_in - BB[:1]) / Res).astype(np.int32)
grid_inbound = ((data_grid >= 0) & (data_grid < np.expand_dims(ObsMask.shape, 0))).sum(axis=-1) ==3
data_grid_in = data_grid[grid_inbound]
in_obs = ObsMask[data_grid_in[:,0], data_grid_in[:,1], data_grid_in[:,2]].astype(np.bool_)
data_in_obs = data_in[grid_inbound][in_obs]

masking data pcd:  25%|████████████████████▌                                                             | 2/8 [00:13<00:41,  6.96s/it]

In [9]:
pbar.update(1)
pbar.set_description('read STL pcd')
stl_pcd = o3d.io.read_point_cloud(f'{args.dataset_dir}/Points/stl/stl{args.scan:03}_total.ply')
stl = np.asarray(stl_pcd.points)

pbar.update(1)
pbar.set_description('compute data2stl')
nn_engine.fit(stl)
dist_d2s, idx_d2s = nn_engine.kneighbors(data_in_obs, n_neighbors=1, return_distance=True)
max_dist = args.max_dist
mean_d2s = dist_d2s[dist_d2s < max_dist].mean()

pbar.update(1)
pbar.set_description('compute stl2data')
ground_plane = loadmat(f'{args.dataset_dir}/ObsMask/Plane{args.scan}.mat')['P']

stl_hom = np.concatenate([stl, np.ones_like(stl[:,:1])], -1)
above = (ground_plane.reshape((1,4)) * stl_hom).sum(-1) > 0
stl_above = stl[above]

nn_engine.fit(data_in)
dist_s2d, idx_s2d = nn_engine.kneighbors(stl_above, n_neighbors=1, return_distance=True)
mean_s2d = dist_s2d[dist_s2d < max_dist].mean()

pbar.update(1)
pbar.set_description('visualize error')
vis_dist = args.visualize_threshold
R = np.array([[1,0,0]], dtype=np.float64)
G = np.array([[0,1,0]], dtype=np.float64)
B = np.array([[0,0,1]], dtype=np.float64)
W = np.array([[1,1,1]], dtype=np.float64)
data_color = np.tile(B, (data_down.shape[0], 1))
data_alpha = dist_d2s.clip(max=vis_dist) / vis_dist
data_color[ np.where(inbound)[0][grid_inbound][in_obs] ] = R * data_alpha + W * (1-data_alpha)
data_color[ np.where(inbound)[0][grid_inbound][in_obs][dist_d2s[:,0] >= max_dist] ] = G
write_vis_pcd(f'{args.pts_dir}/vis_{args.scan:03}_d2s.ply', data_down, data_color)
stl_color = np.tile(B, (stl.shape[0], 1))
stl_alpha = dist_s2d.clip(max=vis_dist) / vis_dist
stl_color[ np.where(above)[0] ] = R * stl_alpha + W * (1-stl_alpha)
stl_color[ np.where(above)[0][dist_s2d[:,0] >= max_dist] ] = G
write_vis_pcd(f'{args.pts_dir}/vis_{args.scan:03}_s2d.ply', stl, stl_color)

pbar.update(1)
pbar.set_description('done')
pbar.close()
over_all = (mean_d2s + mean_s2d) / 2
print(mean_d2s, mean_s2d, over_all)

with open(f'{args.pts_dir}/cf.txt', 'w') as f:
    f.write(f'Mean d2s: {mean_d2s}\n')
    f.write(f'Mean s2d: {mean_s2d}\n')
    f.write(f'Over all: {over_all}\n')



done:  88%|██████████████████████████████████████████████████████████████████████████████████▎           | 7/8 [01:12<00:10, 10.31s/it]

2.9290515172293006 3.4710870208708196 3.20006926905006


In [10]:
plt_points = k3d.points(data_in[0:-1:1,:],
                        color=0xff0000,
                        point_size=0.0001,
                        shader="flat")

plt_points_2 = k3d.points(stl[0:-1:1,:],
                        color=0x00ff00,
                        point_size=0.0001,
                        shader="flat")

plot = k3d.plot(grid_visible=False,
                camera_auto_fit=True)
plot += plt_points
# plot += plt_points_2
plot.display()

/home/tw554/.conda/envs/voxel_sdf/lib/python3.8/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


Output()

In [11]:


plt_points = k3d.points(data_in_obs,
                        color=0xff0000,
                        point_size=0.0001,
                        shader="flat")

plt_points_2 = k3d.points(stl[0:-1:5,:],
                        color=0x00ff00,
                        point_size=0.0001,
                        shader="flat")

plot = k3d.plot(grid_visible=False,
                camera_auto_fit=True)
plot += plt_points
plot += plt_points_2
plot.display()

Output()